In [3]:
# descargamos la biblioteca de nltk
!pip install nltk

In [5]:
# importamos nltk y las partes que nos interezan
import nltk
import random
from nltk import classify, pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.classify.naivebayes import NaiveBayesClassifier

# descargamos los paquetes de nltk para hacerlo mas simple
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [6]:
# agregamos a una variable los archivos descargados
positive = twitter_samples.strings('positive_tweets.json')
negative = twitter_samples.strings('negative_tweets.json')
stop_words = list(set(stopwords.words('english')))

In [ ]:
# con esto podemos ver el contenido de lo que descargamos
print(positive[1])

@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!


In [7]:
# ahora separaremos los contenidos de lo descargado para porder analizarlos por partes
positive_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tokens = twitter_samples.tokenized('negative_tweets.json')

In [ ]:
# aqui podemos ver lo que hicimos en el apartado anterior
print(positive_tokens[1])

['@Lamb2ja', 'Hey', 'James', '!', 'How', 'odd', ':/', 'Please', 'call', 'our', 'Contact', 'Centre', 'on', '02392441234', 'and', 'we', 'will', 'be', 'able', 'to', 'assist', 'you', ':)', 'Many', 'thanks', '!']


In [9]:
# def 1
def clean(tokens):
    tokens = [x for x in tokens if not x in stop_words]

    l = WordNetLemmatizer()
    lemmatized = []

    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized.append(l.lemmatize(word, pos))
    return lemmatized
    
# def 2

positive_clean = []
negative_clean = []

for token in positive_tokens:
    positive_clean.append(clean(token))

for token in negative_tokens:
    negative_clean.append(clean(token))

    def final_token_generator(tokens):
      for tokens in tokens:
        yield dict([token, True] for token in tokens)

# creamos los datasets positivos y negativos

positive_model_tokens = final_token_generator(positive_clean)
negative_model_tokens = final_token_generator(negative_clean)

positive_dataset = [(token, "Positive") for token in positive_model_tokens]
negative_dataset = [(token, "Negative") for token in negative_model_tokens]

In [10]:
# ahora creamos el dataset general que utilizaremos
dataset = positive_dataset + negative_dataset

# movemos un poco y separamos los positivos de los negativos
random.shuffle(dataset)
random.shuffle(dataset)
random.shuffle(dataset)

#separamos el dataset en train y test, sabien ya de antemano que usamos 14000 registros
training = dataset[:7000]
testing = dataset[7000:]

In [11]:
# creamos el clasificador y entrenamos el modelo
classifier = NaiveBayesClassifier.train(training)

In [12]:
#aqui podemos ver la precision de nuestro modelo simple
print("Accuracy:", classify.accuracy(classifier, testing))

Accuracy: 0.9946666666666667


In [13]:
# creamos nuestra funcion para correr el programa
def analyze(input):
    custom_tokens = clean(word_tokenize(input))
    return classifier.classify(dict([token, True] for token in custom_tokens))

In [14]:
# utilizamos el programa y probamos unas cuentas peticiones
analyze("the days are turning darker and darker everyday")

'Negative'

In [15]:
analyze("today i had a beautifull morning")

'Positive'

In [16]:
analyze("I'm so tired of the same thing everyday")

'Negative'

In [17]:
analyze("haha you are so funny")

'Positive'